# Assigning TIFs to Licenses and Communities

In [ ]:
!pip install geopy==2.4.1
!pip install geodatasets==2023.12.0
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import geodatasets
import matplotlib.pyplot as plt
from shapely.geometry import Point
import json


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
df_1 = _deepnote_execute_sql('SELECT *\n    FROM \'BusCorrected.csv\'\n    WHERE STATE = \'IL\' AND CITY = \'CHICAGO\' \n    AND ADDRESS IS NOT NULL \n    AND CITY IS NOT NULL \n    AND STATE IS NOT NULL \n    AND ZIP_CODE IS NOT NULL \n    AND LATITUDE IS NOT NULL \n    AND LONGITUDE IS NOT NULL\n-- LIMIT 1000;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

,ID,LICENSE_ID,ACCOUNT_NUMBER,SITE_NUMBER,LEGAL_NAME,DBA,ADDRESS,CITY,STATE,ZIP_CODE,...,LICENSE_TERM_START_DATE,LICENSE_TERM_EXPIRATION_DATE,LICENSE_APPROVED_FOR_ISSUANCE,DATE_ISSUED,LICENSE_STATUS,LICENSE_STATUS_CHANGE_DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,2617269-20220916,2857496,1611,5,TTX COMPANY,TTX COMPANY,4016 S ASHLAND AVE,CHICAGO,IL,60609,...,2022-09-16,2024-09-15,2022-08-05,2022-08-08,AAI,NaT,NaN,41.820285,-87.665397,"(41.82028493557808, -87.66539682158037)"
1,2652692-20201217,2652692,456833,1,JYM INVESTMENTS LLC,WINGMAN,2321 W HOWARD ST 1ST,CHICAGO,IL,60645,...,2020-12-17,2023-01-15,2020-12-16,2020-12-17,AAC,2022-08-08,NaN,42.019413,-87.688628,"(42.019413270594214, -87.68862818775574)"
2,2647849-20200916,2739788,64603,7,LEOPARDO COMPANIES INC.,Leopardo Companies Inc,210 N CARPENTER ST 3RD FLOOR,CHICAGO,IL,60607,...,2020-09-16,2022-09-15,2020-08-20,2020-08-21,AAC,2022-08-08,NaN,41.885945,-87.653462,"(41.88594495760403, -87.65346178255953)"
3,1579838-20220616,2842924,289144,1,"1021 MONTROSE, L.L.C.",Driftwood,1021 W MONTROSE AVE,CHICAGO,IL,60613,...,2022-06-16,2024-06-15,2022-05-10,2022-05-11,AAI,NaT,34.0,41.961655,-87.655650,"(41.961655308981946, -87.65564994485091)"
4,2027540-20220616,2841687,10726,5,GRAHAM ENTERPRISE INC,Rock N Roll BP (GEI-157),631-647 N LA SALLE DR,CHICAGO,IL,60654,...,2022-06-16,2024-06-15,2022-05-10,2022-05-11,AAI,NaT,NaN,41.893251,-87.632493,"(41.89325119846543, -87.63249301222938)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033611,2204520-20160716,2465534,25985,1,"AABALL HEATING SERVICE & SUPPLIES, INC.",AABALL HEATING SERVICE & SUPPLIES,8518 S ASHLAND AVE,CHICAGO,IL,60620,...,2016-07-16,2018-07-15,2016-05-31,2016-06-01,AAI,NaT,69.0,41.738831,-87.663289,"(41.738831043655715, -87.66328877414871)"
1033612,2652764-20210316,2770633,456868,1,SHOBHNA INC.,FOOD STOP,3734 W LAWRENCE AVE,CHICAGO,IL,60625,...,2021-03-16,2022-03-15,2021-04-02,2021-04-05,AAI,NaT,NaN,41.968417,-87.722060,"(41.96841672465387, -87.7220597187097)"
1033613,6415-20070816,1832401,376290,2,"FRANK'S CHICAGO SHRIMP HOUSE, INC.",FRANK'S CHICAGO SHRIMP HOUSE,5439 W ADDISON ST 1ST,CHICAGO,IL,60641,...,2007-08-16,2009-08-15,2007-08-24,2008-05-22,AAI,NaT,NaN,41.945810,-87.763441,"(41.94580991762778, -87.76344146216435)"
1033614,1800007-20070124,1800007,313705,1,TERRANCE J. WATTS,TERRANCE J. WATTS,9932 S SANGAMON ST,CHICAGO,IL,60643,...,2007-01-24,2009-02-15,NaT,2007-01-24,AAI,NaT,NaN,41.713160,-87.646789,"(41.71316042726137, -87.6467890258547)"


In [ ]:
seen = set()  # Create a set to store unique values
new_list = []

for val in df_1['BUSINESS_ACTIVITY']:
#   print(val)
  if val != None:
    for x in val.split('|'):
        x = x.strip()
        if x not in seen:
            seen.add(x)  # Add unique value to the set
            new_list.append(x)
print(new_list)

['Miscellaneous Commercial Services', 'Sale of Food Prepared Onsite With Dining Area', 'Administrative Commercial Office', 'Tavern - Consumption of Liquor on Premise', 'Operation of a Fuel Filling Station', 'Freelance Writer (Home Based Business)', 'Graphic Design (Home Based Business)', 'Marketing / Advertising / Public Relations (Home Based Business)', 'Professional Consulting (Home Based Business)', 'Photography Services', 'Multi-Media Services', 'Nail Services', 'Provides Onsite Amusement or Entertainment', 'Graphic Design and Consultation', 'Retail Sales of General Merchandise', 'Retail Sales of Perishable Foods', 'Shared Housing Operator - Host of More Than One Shared Housing Unit', 'Retail Sale of Tobacco', 'Consumption of Liquor on Premises', 'Dry Cleaning - Drop Off Location', 'Organizer of a 1-5 Day Event w/ Pop-Up Retail Users at a Trade Show, Exhibition or Convention', 'Catering of Liquor To Events', 'Sale of Vehicle Parts and Accessories', 'Retail Sales of Tobacco Products

In [ ]:
len(seen)

441

In [ ]:
with open('unique_values.txt', 'w') as f:
  f.write('\n'.join(new_list))

In [ ]:
# calling the Nominatim tool and create Nominatim class
loc = Nominatim(user_agent="Geopy Library")

def getLongAlt(address):
    getLoc = loc.geocode(address)
    return getLoc.latitude, getLoc.longitude

In [ ]:
tif_geo = gpd.read_file('Boundaries TIF.geojson')
com_geo = gpd.read_file('Community Areas.geojson')
# Set the coordinate reference system to the standard latitude/longitude system
tif_geo = tif_geo.set_crs(epsg=4326)
com_geo = com_geo.set_crs(epsg=4326)

# tif_geo.plot()
# tif.shape
# len(tif)

In [ ]:
def isInsideTIF_Geo(point):# Point(long, lat)
    # takes a Point class and returns the index of the TIF that contains the Point
    # NOTE: RETURNS AN ARRAY NOT BOOL
    returnArray = []
    # iterate over all the distrcits and see if it contains the given point
    for index, row in tif_geo.iterrows():
        if row['geometry'].contains(point):
            # returnArray.append(index)
            returnArray.append(row['name'])
    
    return returnArray

# long, lat
print(isInsideTIF_Geo(Point(-87.65454023,41.68421355)))


['119th Street/I-57']


In [ ]:
df_1['TIF_Name'] = [isInsideTIF_Geo(Point(row['LONGITUDE'],row['LATITUDE'])) for index, row in df_1.iterrows()]
df_1.to_csv('BusWithTIF.csv', index=False)
df_1

,ID,LICENSE_ID,ACCOUNT_NUMBER,SITE_NUMBER,LEGAL_NAME,DBA,ADDRESS,CITY,STATE,ZIP_CODE,...,LICENSE_TERM_EXPIRATION_DATE,LICENSE_APPROVED_FOR_ISSUANCE,DATE_ISSUED,LICENSE_STATUS,LICENSE_STATUS_CHANGE_DATE,SSA,LATITUDE,LONGITUDE,LOCATION,TIF_Name
0,2617269-20220916,2857496,1611,5,TTX COMPANY,TTX COMPANY,4016 S ASHLAND AVE,CHICAGO,IL,60609,...,2024-09-15,2022-08-05,2022-08-08,AAI,NaT,NaN,41.820285,-87.665397,"(41.82028493557808, -87.66539682158037)",[]
1,2652692-20201217,2652692,456833,1,JYM INVESTMENTS LLC,WINGMAN,2321 W HOWARD ST 1ST,CHICAGO,IL,60645,...,2023-01-15,2020-12-16,2020-12-17,AAC,2022-08-08,NaN,42.019413,-87.688628,"(42.019413270594214, -87.68862818775574)",[]
2,2647849-20200916,2739788,64603,7,LEOPARDO COMPANIES INC.,Leopardo Companies Inc,210 N CARPENTER ST 3RD FLOOR,CHICAGO,IL,60607,...,2022-09-15,2020-08-20,2020-08-21,AAC,2022-08-08,NaN,41.885945,-87.653462,"(41.88594495760403, -87.65346178255953)",[Kinzie Industrial Conservation Area]
3,1579838-20220616,2842924,289144,1,"1021 MONTROSE, L.L.C.",Driftwood,1021 W MONTROSE AVE,CHICAGO,IL,60613,...,2024-06-15,2022-05-10,2022-05-11,AAI,NaT,34.0,41.961655,-87.655650,"(41.961655308981946, -87.65564994485091)",[Wilson Yard]
4,2027540-20220616,2841687,10726,5,GRAHAM ENTERPRISE INC,Rock N Roll BP (GEI-157),631-647 N LA SALLE DR,CHICAGO,IL,60654,...,2024-06-15,2022-05-10,2022-05-11,AAI,NaT,NaN,41.893251,-87.632493,"(41.89325119846543, -87.63249301222938)",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033611,2204520-20160716,2465534,25985,1,"AABALL HEATING SERVICE & SUPPLIES, INC.",AABALL HEATING SERVICE & SUPPLIES,8518 S ASHLAND AVE,CHICAGO,IL,60620,...,2018-07-15,2016-05-31,2016-06-01,AAI,NaT,69.0,41.738831,-87.663289,"(41.738831043655715, -87.66328877414871)",[]
1033612,2652764-20210316,2770633,456868,1,SHOBHNA INC.,FOOD STOP,3734 W LAWRENCE AVE,CHICAGO,IL,60625,...,2022-03-15,2021-04-02,2021-04-05,AAI,NaT,NaN,41.968417,-87.722060,"(41.96841672465387, -87.7220597187097)",[Lawrence/Kedzie]
1033613,6415-20070816,1832401,376290,2,"FRANK'S CHICAGO SHRIMP HOUSE, INC.",FRANK'S CHICAGO SHRIMP HOUSE,5439 W ADDISON ST 1ST,CHICAGO,IL,60641,...,2009-08-15,2007-08-24,2008-05-22,AAI,NaT,NaN,41.945810,-87.763441,"(41.94580991762778, -87.76344146216435)",[]
1033614,1800007-20070124,1800007,313705,1,TERRANCE J. WATTS,TERRANCE J. WATTS,9932 S SANGAMON ST,CHICAGO,IL,60643,...,2009-02-15,NaT,2007-01-24,AAI,NaT,NaN,41.713160,-87.646789,"(41.71316042726137, -87.6467890258547)",[105th Street and Vincennes Avenue]


In [ ]:
df_2 = _deepnote_execute_sql('SELECT *\n    FROM \'BusV3.csv\'\n    WHERE STATE = \'IL\' AND CITY = \'CHICAGO\' \n    AND ADDRESS IS NOT NULL \n    AND CITY IS NOT NULL \n    AND STATE IS NOT NULL \n    AND ZIP_CODE IS NOT NULL \n    AND LATITUDE IS NOT NULL \n    AND LONGITUDE IS NOT NULL\n-- LIMIT 1000;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_2

,ID,LICENSE_ID,ACCOUNT_NUMBER,SITE_NUMBER,LEGAL_NAME,DBA,ADDRESS,CITY,STATE,ZIP_CODE,...,LICENSE_APPROVED_FOR_ISSUANCE,DATE_ISSUED,LICENSE_STATUS,LICENSE_STATUS_CHANGE_DATE,SSA,LATITUDE,LONGITUDE,LOCATION,TIF_Name,LICENSE_GROUP
0,2617269-20220916,2857496,1611,5,TTX COMPANY,TTX COMPANY,4016 S ASHLAND AVE,CHICAGO,IL,60609,...,2022-08-05,2022-08-08,AAI,NaT,NaN,41.820285,-87.665397,"(41.82028493557808, -87.66539682158037)",[],Business
1,2652692-20201217,2652692,456833,1,JYM INVESTMENTS LLC,WINGMAN,2321 W HOWARD ST 1ST,CHICAGO,IL,60645,...,2020-12-16,2020-12-17,AAC,2022-08-08,NaN,42.019413,-87.688628,"(42.019413270594214, -87.68862818775574)",[],Food
2,2647849-20200916,2739788,64603,7,LEOPARDO COMPANIES INC.,Leopardo Companies Inc,210 N CARPENTER ST 3RD FLOOR,CHICAGO,IL,60607,...,2020-08-20,2020-08-21,AAC,2022-08-08,NaN,41.885945,-87.653462,"(41.88594495760403, -87.65346178255953)",['Kinzie Industrial Conservation Area'],Business
3,1579838-20220616,2842924,289144,1,"1021 MONTROSE, L.L.C.",Driftwood,1021 W MONTROSE AVE,CHICAGO,IL,60613,...,2022-05-10,2022-05-11,AAI,NaT,34.0,41.961655,-87.655650,"(41.961655308981946, -87.65564994485091)",['Wilson Yard'],Food
4,2027540-20220616,2841687,10726,5,GRAHAM ENTERPRISE INC,Rock N Roll BP (GEI-157),631-647 N LA SALLE DR,CHICAGO,IL,60654,...,2022-05-10,2022-05-11,AAI,NaT,NaN,41.893251,-87.632493,"(41.89325119846543, -87.63249301222938)",[],Vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033611,2204520-20160716,2465534,25985,1,"AABALL HEATING SERVICE & SUPPLIES, INC.",AABALL HEATING SERVICE & SUPPLIES,8518 S ASHLAND AVE,CHICAGO,IL,60620,...,2016-05-31,2016-06-01,AAI,NaT,69.0,41.738831,-87.663289,"(41.738831043655715, -87.66328877414871)",[],Business
1033612,2652764-20210316,2770633,456868,1,SHOBHNA INC.,FOOD STOP,3734 W LAWRENCE AVE,CHICAGO,IL,60625,...,2021-04-02,2021-04-05,AAI,NaT,NaN,41.968417,-87.722060,"(41.96841672465387, -87.7220597187097)",['Lawrence/Kedzie'],Tabacco
1033613,6415-20070816,1832401,376290,2,"FRANK'S CHICAGO SHRIMP HOUSE, INC.",FRANK'S CHICAGO SHRIMP HOUSE,5439 W ADDISON ST 1ST,CHICAGO,IL,60641,...,2007-08-24,2008-05-22,AAI,NaT,NaN,41.945810,-87.763441,"(41.94580991762778, -87.76344146216435)",[],Food
1033614,1800007-20070124,1800007,313705,1,TERRANCE J. WATTS,TERRANCE J. WATTS,9932 S SANGAMON ST,CHICAGO,IL,60643,...,NaT,2007-01-24,AAI,NaT,NaN,41.713160,-87.646789,"(41.71316042726137, -87.6467890258547)",['105th Street and Vincennes Avenue'],Special


In [ ]:
def isInsideCom_Geo(point, tif_list):# Point(long, lat)
    # takes a Point class and returns the index of the TIF that contains the Point
    # NOTE: RETURNS AN ARRAY NOT BOOL
    returnArray = []

    # iterate over all the distrcits and see if it contains the given point
    # print(len(tif_list) ,tif_list)
    if len(tif_list) == 2:
        # print("oo")
        for index, row in com_geo.iterrows():
            if row['geometry'].contains(point):
                # returnArray.append(index)
                returnArray.append(row['community'])
    
    return returnArray

# long, lat
print(isInsideCom_Geo(Point(-87.65454023,41.68421355), '[]'))

['WEST PULLMAN']


In [ ]:
df_2['Com_Name'] = [isInsideCom_Geo(Point(row['LONGITUDE'],row['LATITUDE']), row['TIF_Name']) for index, row in df_2.iterrows()]
df_2.to_csv('BusWithCom.csv', index=False)
df_2

,ID,LICENSE_ID,ACCOUNT_NUMBER,SITE_NUMBER,LEGAL_NAME,DBA,ADDRESS,CITY,STATE,ZIP_CODE,...,DATE_ISSUED,LICENSE_STATUS,LICENSE_STATUS_CHANGE_DATE,SSA,LATITUDE,LONGITUDE,LOCATION,TIF_Name,LICENSE_GROUP,Com_Name
0,2617269-20220916,2857496,1611,5,TTX COMPANY,TTX COMPANY,4016 S ASHLAND AVE,CHICAGO,IL,60609,...,2022-08-08,AAI,NaT,NaN,41.820285,-87.665397,"(41.82028493557808, -87.66539682158037)",[],Business,[NEW CITY]
1,2652692-20201217,2652692,456833,1,JYM INVESTMENTS LLC,WINGMAN,2321 W HOWARD ST 1ST,CHICAGO,IL,60645,...,2020-12-17,AAC,2022-08-08,NaN,42.019413,-87.688628,"(42.019413270594214, -87.68862818775574)",[],Food,[WEST RIDGE]
2,2647849-20200916,2739788,64603,7,LEOPARDO COMPANIES INC.,Leopardo Companies Inc,210 N CARPENTER ST 3RD FLOOR,CHICAGO,IL,60607,...,2020-08-21,AAC,2022-08-08,NaN,41.885945,-87.653462,"(41.88594495760403, -87.65346178255953)",['Kinzie Industrial Conservation Area'],Business,[]
3,1579838-20220616,2842924,289144,1,"1021 MONTROSE, L.L.C.",Driftwood,1021 W MONTROSE AVE,CHICAGO,IL,60613,...,2022-05-11,AAI,NaT,34.0,41.961655,-87.655650,"(41.961655308981946, -87.65564994485091)",['Wilson Yard'],Food,[]
4,2027540-20220616,2841687,10726,5,GRAHAM ENTERPRISE INC,Rock N Roll BP (GEI-157),631-647 N LA SALLE DR,CHICAGO,IL,60654,...,2022-05-11,AAI,NaT,NaN,41.893251,-87.632493,"(41.89325119846543, -87.63249301222938)",[],Vehicles,[NEAR NORTH SIDE]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033611,2204520-20160716,2465534,25985,1,"AABALL HEATING SERVICE & SUPPLIES, INC.",AABALL HEATING SERVICE & SUPPLIES,8518 S ASHLAND AVE,CHICAGO,IL,60620,...,2016-06-01,AAI,NaT,69.0,41.738831,-87.663289,"(41.738831043655715, -87.66328877414871)",[],Business,[AUBURN GRESHAM]
1033612,2652764-20210316,2770633,456868,1,SHOBHNA INC.,FOOD STOP,3734 W LAWRENCE AVE,CHICAGO,IL,60625,...,2021-04-05,AAI,NaT,NaN,41.968417,-87.722060,"(41.96841672465387, -87.7220597187097)",['Lawrence/Kedzie'],Tabacco,[]
1033613,6415-20070816,1832401,376290,2,"FRANK'S CHICAGO SHRIMP HOUSE, INC.",FRANK'S CHICAGO SHRIMP HOUSE,5439 W ADDISON ST 1ST,CHICAGO,IL,60641,...,2008-05-22,AAI,NaT,NaN,41.945810,-87.763441,"(41.94580991762778, -87.76344146216435)",[],Food,[PORTAGE PARK]
1033614,1800007-20070124,1800007,313705,1,TERRANCE J. WATTS,TERRANCE J. WATTS,9932 S SANGAMON ST,CHICAGO,IL,60643,...,2007-01-24,AAI,NaT,NaN,41.713160,-87.646789,"(41.71316042726137, -87.6467890258547)",['105th Street and Vincennes Avenue'],Special,[]


In [ ]:
df_3 = _deepnote_execute_sql('SELECT *\n    FROM \'BusWithCom.csv\'\n    WHERE STATE = \'IL\' AND CITY = \'CHICAGO\' \n    AND ADDRESS IS NOT NULL \n    AND CITY IS NOT NULL \n    AND STATE IS NOT NULL \n    AND ZIP_CODE IS NOT NULL \n    AND LATITUDE IS NOT NULL \n    AND LONGITUDE IS NOT NULL', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_3

,ID,LICENSE_ID,ACCOUNT_NUMBER,SITE_NUMBER,LEGAL_NAME,DBA,ADDRESS,CITY,STATE,ZIP_CODE,...,DATE_ISSUED,LICENSE_STATUS,LICENSE_STATUS_CHANGE_DATE,SSA,LATITUDE,LONGITUDE,LOCATION,TIF_Name,LICENSE_GROUP,Com_Name
0,2617269-20220916,2857496,1611,5,TTX COMPANY,TTX COMPANY,4016 S ASHLAND AVE,CHICAGO,IL,60609,...,2022-08-08,AAI,NaT,NaN,41.820285,-87.665397,"(41.82028493557808, -87.66539682158037)",[],Business,['NEW CITY']
1,2652692-20201217,2652692,456833,1,JYM INVESTMENTS LLC,WINGMAN,2321 W HOWARD ST 1ST,CHICAGO,IL,60645,...,2020-12-17,AAC,2022-08-08,NaN,42.019413,-87.688628,"(42.019413270594214, -87.68862818775574)",[],Food,['WEST RIDGE']
2,2647849-20200916,2739788,64603,7,LEOPARDO COMPANIES INC.,Leopardo Companies Inc,210 N CARPENTER ST 3RD FLOOR,CHICAGO,IL,60607,...,2020-08-21,AAC,2022-08-08,NaN,41.885945,-87.653462,"(41.88594495760403, -87.65346178255953)",['Kinzie Industrial Conservation Area'],Business,[]
3,1579838-20220616,2842924,289144,1,"1021 MONTROSE, L.L.C.",Driftwood,1021 W MONTROSE AVE,CHICAGO,IL,60613,...,2022-05-11,AAI,NaT,34.0,41.961655,-87.655650,"(41.961655308981946, -87.65564994485091)",['Wilson Yard'],Food,[]
4,2027540-20220616,2841687,10726,5,GRAHAM ENTERPRISE INC,Rock N Roll BP (GEI-157),631-647 N LA SALLE DR,CHICAGO,IL,60654,...,2022-05-11,AAI,NaT,NaN,41.893251,-87.632493,"(41.89325119846543, -87.63249301222938)",[],Vehicles,['NEAR NORTH SIDE']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033611,2204520-20160716,2465534,25985,1,"AABALL HEATING SERVICE & SUPPLIES, INC.",AABALL HEATING SERVICE & SUPPLIES,8518 S ASHLAND AVE,CHICAGO,IL,60620,...,2016-06-01,AAI,NaT,69.0,41.738831,-87.663289,"(41.738831043655715, -87.66328877414871)",[],Business,['AUBURN GRESHAM']
1033612,2652764-20210316,2770633,456868,1,SHOBHNA INC.,FOOD STOP,3734 W LAWRENCE AVE,CHICAGO,IL,60625,...,2021-04-05,AAI,NaT,NaN,41.968417,-87.722060,"(41.96841672465387, -87.7220597187097)",['Lawrence/Kedzie'],Tabacco,[]
1033613,6415-20070816,1832401,376290,2,"FRANK'S CHICAGO SHRIMP HOUSE, INC.",FRANK'S CHICAGO SHRIMP HOUSE,5439 W ADDISON ST 1ST,CHICAGO,IL,60641,...,2008-05-22,AAI,NaT,NaN,41.945810,-87.763441,"(41.94580991762778, -87.76344146216435)",[],Food,['PORTAGE PARK']
1033614,1800007-20070124,1800007,313705,1,TERRANCE J. WATTS,TERRANCE J. WATTS,9932 S SANGAMON ST,CHICAGO,IL,60643,...,2007-01-24,AAI,NaT,NaN,41.713160,-87.646789,"(41.71316042726137, -87.6467890258547)",['105th Street and Vincennes Avenue'],Special,[]


In [ ]:
import pandas as pd
crossTab_percent_df = pd.crosstab(index=df_3['Com_Name'], columns=df_3['LICENSE_GROUP'], normalize='index')
crossTab_percent_df                                                                                                                                                                                                              

LICENSE_GROUP,Accommodation,Business,Electronic,Entertainment,Food,Liquor,Medical,Other,Permits,Personal Services,Retail,Special,Tabacco,Vehicles
Com_Name,,,,,,,,,,,,,,
['ALBANY PARK'],0.000000,0.436161,0.001621,0.008824,0.199892,0.000180,0.000000,0.000000,0.013326,0.014947,0.034396,0.197011,0.054385,0.039258
['ARCHER HEIGHTS'],0.003371,0.351685,0.003745,0.026217,0.309738,0.000000,0.000000,0.005243,0.002996,0.004494,0.031461,0.123596,0.051685,0.085768
['ARMOUR SQUARE'],0.000000,0.284211,0.000000,0.017544,0.343860,0.049123,0.000000,0.000000,0.000000,0.000000,0.126316,0.140351,0.038596,0.000000
['ASHBURN'],0.001747,0.420271,0.005242,0.009611,0.183486,0.000437,0.000000,0.004369,0.011359,0.006553,0.025994,0.220183,0.047401,0.063346
['AUBURN GRESHAM'],0.001189,0.424112,0.001933,0.008473,0.178534,0.000000,0.000000,0.001041,0.031961,0.004757,0.018879,0.199792,0.072098,0.057232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
['WEST PULLMAN'],0.000000,0.270478,0.000000,0.010239,0.132253,0.000000,0.000000,0.006826,0.023891,0.005119,0.027304,0.393345,0.075939,0.054608
['WEST RIDGE'],0.000824,0.480566,0.003022,0.007142,0.230257,0.000481,0.000000,0.002610,0.013940,0.010301,0.027812,0.119970,0.063315,0.039761
['WEST TOWN'],0.001286,0.432025,0.001829,0.023173,0.236585,0.008772,0.000514,0.002914,0.007086,0.016972,0.037802,0.157066,0.050374,0.023601


In [ ]:
import pandas as pd
crossTab_df = pd.crosstab(index=df_3['Com_Name'], columns=df_3['LICENSE_GROUP'])
crossTab_df

LICENSE_GROUP,Accommodation,Business,Electronic,Entertainment,Food,Liquor,Medical,Other,Permits,Personal Services,Retail,Special,Tabacco,Vehicles
Com_Name,,,,,,,,,,,,,,
['ALBANY PARK'],0,2422,9,49,1110,1,0,0,74,83,191,1094,302,218
['ARCHER HEIGHTS'],9,939,10,70,827,0,0,14,8,12,84,330,138,229
['ARMOUR SQUARE'],0,81,0,5,98,14,0,0,0,0,36,40,11,0
['ASHBURN'],8,1924,24,44,840,2,0,20,52,30,119,1008,217,290
['AUBURN GRESHAM'],8,2853,13,57,1201,0,0,7,215,32,127,1344,485,385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
['WEST PULLMAN'],0,317,0,12,155,0,0,8,28,6,32,461,89,64
['WEST RIDGE'],12,6998,44,104,3353,7,0,38,203,150,405,1747,922,579
['WEST TOWN'],45,15120,64,811,8280,307,18,102,248,594,1323,5497,1763,826


In [ ]:
crossTab_percent_df.to_csv("Community_cross_percent.csv")
crossTab_df.to_csv("Community_cross.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=41bd1eeb-8ade-41a5-ae81-cf4a88f38197' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>